In [1]:
import os
import copy
import torch
import torch.nn as nn
import pandas as pd
from PIL import Image
import cv2 as cv
from google.colab.patches import cv2_imshow
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, models
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Creat datasets and data loaders for image classification purpose

In [2]:
%cd /content/drive/My Drive/CV_Proj/CV21_AnnotatedDataset/augmented_img_7

/content/drive/My Drive/CV_Proj/CV21_AnnotatedDataset/augmented_img_7


In [4]:
strToLabel = {'Chaetomium_Einzelspore_Candidate': 0,'Chaetomium_Agglomerate_Candidate': 1,'Stachybotrys_Einzelspore_Candidate': 2,'Stachybotrys_Agglomerate_Candidate': 3}
labels = [strToLabel['_'.join(t.split('_')[:-1])] for t in os.listdir('./')]

In [5]:
len(labels)

44773

In [6]:
!nvidia-smi

Thu Dec 23 06:24:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
class SporeImageClassificationDataset(Dataset):
    """Spore Images for classification dataset."""

    def __init__(self, root_dir, transform=None):
      self.root_dir = root_dir
      self.img_list = os.listdir(self.root_dir)
      self.transform = transform
      self.label_list = ['Chaetomium_Einzelspore_Candidate','Chaetomium_Agglomerate_Candidate','Stachybotrys_Einzelspore_Candidate','Stachybotrys_Agglomerate_Candidate']

    def __len__(self):
      return len(self.img_list)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.img_list[idx])
        image = Image.open(img_name)
        label = self.label_list.index('_'.join(self.img_list[idx].split('_')[:-1]))


        if self.transform:
            image = self.transform(image)

        return [image, label] 

In [8]:
data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [9]:
train_set = SporeImageClassificationDataset('./', data_transform)

# Load pretrained models

## ResNet

In [21]:
resnet50 = models.resnet50(pretrained=True)
ct = 0
for child in resnet50.children():
  ct += 1
  if ct < 10:
    for param in child.parameters():
        param.requires_grad = False

num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 4)
resnet50 = resnet50.cuda()

## VGG

In [22]:
vgg16 = models.vgg16(pretrained=True)
ct = 0
for child in vgg16.children():
  ct += 1
  if ct < 3:
    for param in child.parameters():
      param.requires_grad = False

num_ftrs = vgg16.classifier[-1].in_features
vgg16.classifier[-1] = nn.Linear(num_ftrs,4)
vgg16 = vgg16.cuda()

## DenseNet

In [11]:
densenet169 = models.densenet169(pretrained=True)
ct = 0
for child in densenet169.children():
  ct += 1
  if ct < 2:
    for param in child.parameters():
      param.requires_grad = False

num_ftrs = densenet169.classifier.in_features
densenet169.classifier = nn.Linear(num_ftrs,4)
densenet169 = densenet169.cuda()

## MobileNet

In [13]:
mobilenet_v2 = models.mobilenet_v2(pretrained=True)
ct = 0
for child in mobilenet_v2.children():
  ct += 1
  if ct < 2:
    for param in child.parameters():
      param.requires_grad = False

num_ftrs = mobilenet_v2.classifier[1].in_features
mobilenet_v2.classifier[1] = nn.Linear(num_ftrs,4)
mobilenet_v2 = mobilenet_v2.cuda()

# Training pretrained models

## Training ResNet

In [ ]:
def reset_weights(m):
  for child in m.children():
    does_grad = False
    for param in child.parameters():
      if param.requires_grad == True:
        does_grad = True
        break
    if hasattr(child, 'reset_parameters') and does_grad:
      print(f'Reset trainable parameters of layer = {child}')
      child.reset_parameters()

In [ ]:
k_folds = 5
num_epochs = 5
loss_function = nn.CrossEntropyLoss()

results = {}

torch.manual_seed(42)

kfold = KFold(n_splits=k_folds, shuffle=True)
  
print('--------------------------------')

for fold, (train_ids, test_ids) in enumerate(kfold.split(train_set)):
  
  print(f'FOLD {fold}')
  print('--------------------------------')
  
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  trainloader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=64, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    train_set,
                    batch_size=128, sampler=test_subsampler)
  
  resnet50.apply(reset_weights)
  
  optimizer = torch.optim.Adam(resnet50.parameters(), lr=5e-4)
  
  for epoch in range(0, num_epochs):

    print(f'Starting epoch {epoch+1}')

    current_loss = 0.0

    print(len(trainloader), len(testloader))

    for i, data in enumerate(trainloader, 0):
      
      inputs, targets = data
      optimizer.zero_grad()
      
      outputs = resnet50(inputs)

      loss = loss_function(outputs, targets)
      loss.backward()
      
      optimizer.step()
      
      current_loss += loss.item()
      if i % 5 == 4:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 5))
          current_loss = 0.0

          
  print('Training process has finished. Saving trained model.')

  print('Starting testing')
  
  save_path = f'../resnet50-fold-{fold}.pth'
  torch.save(resnet50.state_dict(), save_path)

  correct, total = 0, 0
  with torch.no_grad():

    for i, data in enumerate(testloader, 0):

      inputs, targets = data

      outputs = resnet50(inputs)

      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)
  
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} %')
  sum += value
print(f'Average: {sum/len(results.items())} %')

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Linear(in_features=2048, out_features=4, bias=True)
Starting epoch 1
30 4


KeyboardInterrupt: ignored

In [22]:
torch.manual_seed(42)

class_sample_count = np.array(
    [labels.count(t) for t in np.unique(labels)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in labels])
samples_weight = torch.from_numpy(samples_weight)
samples_weight = samples_weight.double()
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight))

trainloader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=128, sampler=sampler, num_workers=8)

optimizer = torch.optim.SGD(resnet50.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.1)
loss_function = nn.CrossEntropyLoss()

num_epochs = 2
for epoch in range(0, num_epochs):

  print(f'Starting epoch {epoch+1}')

  current_loss = 0.0
  current_acc = 0.0
  current_acc_0 = 0.0
  current_acc_1 = 0.0
  current_acc_2 = 0.0
  current_acc_3 = 0.0

  print(len(trainloader))
  for i, data in enumerate(trainloader, 0):
    
    inputs, targets = data
    optimizer.zero_grad()
    
    outputs = resnet50(inputs.cuda())

    # print(targets)
    # print(torch.max(outputs, 1)[1])
    loss = loss_function(outputs, targets.cuda())
    loss.backward()
    optimizer.step()
    
    current_loss += loss.item()

    with torch.no_grad():
      current_acc += sum(targets == torch.argmax(outputs.cpu(), dim=1)) / len(targets)
      current_acc_0 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1) == targets] == 0) / (len(targets[targets == 0]) + 1e-5)
      current_acc_1 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 1) / (len(targets[targets == 1])+ 1e-5)
      current_acc_2 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 2) / (len(targets[targets == 2])+ 1e-5)
      current_acc_3 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 3) / (len(targets[targets == 3])+ 1e-5)

    if i % 5 == 4:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 5))
        
        print('Accuracy after mini-batch %5d: %.3f' %
              (i + 1, current_acc / 5))
        print('Accuracy for class 0 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_0 / 5))
        print('Accuracy for class 1 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_1 / 5))
        print('Accuracy for class 2 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_2 / 5))
        print('Accuracy for class 3 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_3 / 5))

        current_loss = 0.0
        current_acc = 0.0
        current_acc_0 = 0.0
        current_acc_1 = 0.0
        current_acc_2 = 0.0
        current_acc_3 = 0.0

print('Training process has finished. Saving trained model.')

save_path = f'../resnet50.pth'
torch.save(resnet50.state_dict(), save_path)

Starting epoch 1
350
Loss after mini-batch     5: 1.424
Accuracy after mini-batch     5: 0.234
Accuracy for class 0 after mini-batch     5: 0.418
Accuracy for class 1 after mini-batch     5: 0.015
Accuracy for class 2 after mini-batch     5: 0.060
Accuracy for class 3 after mini-batch     5: 0.442
Loss after mini-batch    10: 1.372
Accuracy after mini-batch    10: 0.323
Accuracy for class 0 after mini-batch    10: 0.078
Accuracy for class 1 after mini-batch    10: 0.241
Accuracy for class 2 after mini-batch    10: 0.411
Accuracy for class 3 after mini-batch    10: 0.565
Loss after mini-batch    15: 1.289
Accuracy after mini-batch    15: 0.464
Accuracy for class 0 after mini-batch    15: 0.457
Accuracy for class 1 after mini-batch    15: 0.796
Accuracy for class 2 after mini-batch    15: 0.260
Accuracy for class 3 after mini-batch    15: 0.331
Loss after mini-batch    20: 1.222
Accuracy after mini-batch    20: 0.542
Accuracy for class 0 after mini-batch    20: 0.782
Accuracy for class 1

## Training VGG-16

In [ ]:
def reset_weights(m):
  for child in m.children():
    does_grad = False
    for param in child.parameters():
      if param.requires_grad == True:
        does_grad = True
        break
    if hasattr(child, 'reset_parameters') and does_grad:
      print(f'Reset trainable parameters of layer = {child}')
      child.reset_parameters()

In [ ]:
k_folds = 5
num_epochs = 5
loss_function = nn.CrossEntropyLoss()

results = {}

torch.manual_seed(42)

kfold = KFold(n_splits=k_folds, shuffle=True)
  
print('--------------------------------')

for fold, (train_ids, test_ids) in enumerate(kfold.split(train_set)):
  
  print(f'FOLD {fold}')
  print('--------------------------------')
  
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  trainloader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=64, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    train_set,
                    batch_size=128, sampler=test_subsampler)
  
  vgg16.apply(reset_weights)
  
  optimizer = torch.optim.Adam(vgg16.parameters(), lr=5e-4)
  
  for epoch in range(0, num_epochs):

    print(f'Starting epoch {epoch+1}')

    current_loss = 0.0

    for i, data in enumerate(trainloader, 0):
      
      inputs, targets = data
      optimizer.zero_grad()
      
      outputs = vgg16(inputs)

      loss = loss_function(outputs, targets)
      loss.backward()
      
      optimizer.step()
      
      current_loss += loss.item()
      if i % 5 == 4:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 5))
          current_loss = 0.0

          
  print('Training process has finished. Saving trained model.')

  print('Starting testing')
  
  save_path = f'../rvgg16-fold-{fold}.pth'
  torch.save(vgg16.state_dict(), save_path)

  correct, total = 0, 0
  with torch.no_grad():

    for i, data in enumerate(testloader, 0):

      inputs, targets = data

      outputs = vgg16(inputs)

      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)
  
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} %')
  sum += value
print(f'Average: {sum/len(results.items())} %')

--------------------------------
FOLD 0
--------------------------------
Reset trainable parameters of layer = Linear(in_features=25088, out_features=4096, bias=True)
Reset trainable parameters of layer = Linear(in_features=4096, out_features=4096, bias=True)
Reset trainable parameters of layer = Linear(in_features=4096, out_features=4, bias=True)
Starting epoch 1


KeyboardInterrupt: ignored

In [23]:
torch.manual_seed(42)

class_sample_count = np.array(
    [labels.count(t) for t in np.unique(labels)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in labels])
samples_weight = torch.from_numpy(samples_weight)
samples_weight = samples_weight.double()
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight))

trainloader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=128, sampler=sampler, num_workers=8)

optimizer = torch.optim.SGD(vgg16.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.1)
loss_function = nn.CrossEntropyLoss()

num_epochs = 2
for epoch in range(0, num_epochs):

  print(f'Starting epoch {epoch+1}')

  current_loss = 0.0
  current_acc = 0.0
  current_acc_0 = 0.0
  current_acc_1 = 0.0
  current_acc_2 = 0.0
  current_acc_3 = 0.0

  print(len(trainloader))
  for i, data in enumerate(trainloader, 0):
    
    inputs, targets = data
    optimizer.zero_grad()
    
    outputs = vgg16(inputs.cuda())

    # print(targets)
    # print(torch.max(outputs, 1)[1])
    loss = loss_function(outputs, targets.cuda())
    loss.backward()
    optimizer.step()
    
    current_loss += loss.item()

    with torch.no_grad():
      current_acc += sum(targets == torch.argmax(outputs.cpu(), dim=1)) / len(targets)
      current_acc_0 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1) == targets] == 0) / (len(targets[targets == 0]) + 1e-5)
      current_acc_1 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 1) / (len(targets[targets == 1])+ 1e-5)
      current_acc_2 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 2) / (len(targets[targets == 2])+ 1e-5)
      current_acc_3 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 3) / (len(targets[targets == 3])+ 1e-5)

    if i % 5 == 4:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 5))
        
        print('Accuracy after mini-batch %5d: %.3f' %
              (i + 1, current_acc / 5))
        print('Accuracy for class 0 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_0 / 5))
        print('Accuracy for class 1 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_1 / 5))
        print('Accuracy for class 2 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_2 / 5))
        print('Accuracy for class 3 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_3 / 5))

        current_loss = 0.0
        current_acc = 0.0
        current_acc_0 = 0.0
        current_acc_1 = 0.0
        current_acc_2 = 0.0
        current_acc_3 = 0.0

print('Training process has finished. Saving trained model.')

save_path = f'../vgg16.pth'
torch.save(vgg16.state_dict(), save_path)

Starting epoch 1
350
Loss after mini-batch     5: 1.384
Accuracy after mini-batch     5: 0.316
Accuracy for class 0 after mini-batch     5: 0.399
Accuracy for class 1 after mini-batch     5: 0.266
Accuracy for class 2 after mini-batch     5: 0.340
Accuracy for class 3 after mini-batch     5: 0.270
Loss after mini-batch    10: 1.138
Accuracy after mini-batch    10: 0.522
Accuracy for class 0 after mini-batch    10: 0.551
Accuracy for class 1 after mini-batch    10: 0.614
Accuracy for class 2 after mini-batch    10: 0.397
Accuracy for class 3 after mini-batch    10: 0.519
Loss after mini-batch    15: 0.912
Accuracy after mini-batch    15: 0.648
Accuracy for class 0 after mini-batch    15: 0.838
Accuracy for class 1 after mini-batch    15: 0.855
Accuracy for class 2 after mini-batch    15: 0.595
Accuracy for class 3 after mini-batch    15: 0.293
Loss after mini-batch    20: 0.848
Accuracy after mini-batch    20: 0.666
Accuracy for class 0 after mini-batch    20: 0.641
Accuracy for class 1

## Training DenseNet

In [ ]:
k_folds = 5
num_epochs = 5
loss_function = nn.CrossEntropyLoss()

results = {}

torch.manual_seed(42)

kfold = KFold(n_splits=k_folds, shuffle=True)
  
print('--------------------------------')

for fold, (train_ids, test_ids) in enumerate(kfold.split(train_set)):
  
  print(f'FOLD {fold}')
  print('--------------------------------')
  
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  trainloader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=64, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                    train_set,
                    batch_size=128, sampler=test_subsampler)
  
  densenet169.apply(reset_weights)
  
  optimizer = torch.optim.Adam(densenet169.parameters(), lr=5e-4)
  
  for epoch in range(0, num_epochs):

    print(f'Starting epoch {epoch+1}')

    current_loss = 0.0

    print(len(trainloader),len(testloader))
    for i, data in enumerate(trainloader, 0):
      
      inputs, targets = data
      optimizer.zero_grad()
      
      outputs, x = densenet169(inputs)

      loss = loss_function(outputs, targets)
      loss.backward()
      
      optimizer.step()
      
      current_loss += loss.item()
      if i % 5 == 4:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 5))
          current_loss = 0.0

          
  print('Training process has finished. Saving trained model.')

  print('Starting testing')
  
  save_path = f'../densenet169-fold-{fold}.pth'
  torch.save(densenet169.state_dict(), save_path)

  correct, total = 0, 0
  with torch.no_grad():

    for i, data in enumerate(testloader, 0):

      inputs, targets = data

      outputs, x = densenet169(inputs)

      _, predicted = torch.max(outputs.data, 1)
      total += targets.size(0)
      correct += (predicted == targets).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)
  
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} %')
  sum += value
print(f'Average: {sum/len(results.items())} %')

In [12]:
torch.manual_seed(42)

class_sample_count = np.array(
    [labels.count(t) for t in np.unique(labels)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in labels])
samples_weight = torch.from_numpy(samples_weight)
samples_weight = samples_weight.double()
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight))

trainloader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=128, sampler=sampler, num_workers=8)

optimizer = torch.optim.SGD(densenet169.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.1)
loss_function = nn.CrossEntropyLoss()

num_epochs = 2
for epoch in range(0, num_epochs):

  print(f'Starting epoch {epoch+1}')

  current_loss = 0.0
  current_acc = 0.0
  current_acc_0 = 0.0
  current_acc_1 = 0.0
  current_acc_2 = 0.0
  current_acc_3 = 0.0

  print(len(trainloader))
  for i, data in enumerate(trainloader, 0):
    
    inputs, targets = data
    optimizer.zero_grad()
    
    outputs = densenet169(inputs.cuda())

    # print(targets)
    # print(torch.max(outputs, 1)[1])
    loss = loss_function(outputs, targets.cuda())
    loss.backward()
    optimizer.step()
    
    current_loss += loss.item()

    with torch.no_grad():
      current_acc += sum(targets == torch.argmax(outputs.cpu(), dim=1)) / len(targets)
      current_acc_0 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1) == targets] == 0) / (len(targets[targets == 0]) + 1e-5)
      current_acc_1 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 1) / (len(targets[targets == 1])+ 1e-5)
      current_acc_2 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 2) / (len(targets[targets == 2])+ 1e-5)
      current_acc_3 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 3) / (len(targets[targets == 3])+ 1e-5)

    if i % 5 == 4:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 5))
        
        print('Accuracy after mini-batch %5d: %.3f' %
              (i + 1, current_acc / 5))
        print('Accuracy for class 0 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_0 / 5))
        print('Accuracy for class 1 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_1 / 5))
        print('Accuracy for class 2 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_2 / 5))
        print('Accuracy for class 3 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_3 / 5))

        current_loss = 0.0
        current_acc = 0.0
        current_acc_0 = 0.0
        current_acc_1 = 0.0
        current_acc_2 = 0.0
        current_acc_3 = 0.0

print('Training process has finished. Saving trained model.')

save_path = f'../densenet169.pth'
torch.save(densenet169.state_dict(), save_path)

Starting epoch 1
350
Loss after mini-batch     5: 1.404
Accuracy after mini-batch     5: 0.286
Accuracy for class 0 after mini-batch     5: 0.346
Accuracy for class 1 after mini-batch     5: 0.137
Accuracy for class 2 after mini-batch     5: 0.030
Accuracy for class 3 after mini-batch     5: 0.627
Loss after mini-batch    10: 1.366
Accuracy after mini-batch    10: 0.311
Accuracy for class 0 after mini-batch    10: 0.236
Accuracy for class 1 after mini-batch    10: 0.147
Accuracy for class 2 after mini-batch    10: 0.504
Accuracy for class 3 after mini-batch    10: 0.363
Loss after mini-batch    15: 1.296
Accuracy after mini-batch    15: 0.428
Accuracy for class 0 after mini-batch    15: 0.478
Accuracy for class 1 after mini-batch    15: 0.491
Accuracy for class 2 after mini-batch    15: 0.544
Accuracy for class 3 after mini-batch    15: 0.219
Loss after mini-batch    20: 1.237
Accuracy after mini-batch    20: 0.492
Accuracy for class 0 after mini-batch    20: 0.625
Accuracy for class 1

## Training MobileNetv2

In [14]:
torch.manual_seed(42)

class_sample_count = np.array(
    [labels.count(t) for t in np.unique(labels)])
weight = 1. / class_sample_count
samples_weight = np.array([weight[t] for t in labels])
samples_weight = torch.from_numpy(samples_weight)
samples_weight = samples_weight.double()
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight))

trainloader = torch.utils.data.DataLoader(
                    train_set, 
                    batch_size=128, sampler=sampler, num_workers=8)

optimizer = torch.optim.SGD(mobilenet_v2.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.1)
loss_function = nn.CrossEntropyLoss()

num_epochs = 2
for epoch in range(0, num_epochs):

  print(f'Starting epoch {epoch+1}')

  current_loss = 0.0
  current_acc = 0.0
  current_acc_0 = 0.0
  current_acc_1 = 0.0
  current_acc_2 = 0.0
  current_acc_3 = 0.0

  print(len(trainloader))
  for i, data in enumerate(trainloader, 0):
    
    inputs, targets = data
    optimizer.zero_grad()
    
    outputs = mobilenet_v2(inputs.cuda())

    # print(targets)
    # print(torch.max(outputs, 1)[1])
    loss = loss_function(outputs, targets.cuda())
    loss.backward()
    optimizer.step()
    
    current_loss += loss.item()

    with torch.no_grad():
      current_acc += sum(targets == torch.argmax(outputs.cpu(), dim=1)) / len(targets)
      current_acc_0 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1) == targets] == 0) / (len(targets[targets == 0]) + 1e-5)
      current_acc_1 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 1) / (len(targets[targets == 1])+ 1e-5)
      current_acc_2 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 2) / (len(targets[targets == 2])+ 1e-5)
      current_acc_3 += sum(torch.argmax(outputs.cpu(), dim=1)[torch.argmax(outputs.cpu(), dim=1)  == targets] == 3) / (len(targets[targets == 3])+ 1e-5)

    if i % 5 == 4:
        print('Loss after mini-batch %5d: %.3f' %
              (i + 1, current_loss / 5))
        
        print('Accuracy after mini-batch %5d: %.3f' %
              (i + 1, current_acc / 5))
        print('Accuracy for class 0 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_0 / 5))
        print('Accuracy for class 1 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_1 / 5))
        print('Accuracy for class 2 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_2 / 5))
        print('Accuracy for class 3 after mini-batch %5d: %.3f' %
              (i + 1, current_acc_3 / 5))

        current_loss = 0.0
        current_acc = 0.0
        current_acc_0 = 0.0
        current_acc_1 = 0.0
        current_acc_2 = 0.0
        current_acc_3 = 0.0

print('Training process has finished. Saving trained model.')

save_path = f'../mobileNetv2.pth'
torch.save(resnet50.state_dict(), save_path)

Starting epoch 1
350
Loss after mini-batch     5: 1.450
Accuracy after mini-batch     5: 0.262
Accuracy for class 0 after mini-batch     5: 0.574
Accuracy for class 1 after mini-batch     5: 0.170
Accuracy for class 2 after mini-batch     5: 0.054
Accuracy for class 3 after mini-batch     5: 0.285
Loss after mini-batch    10: 1.362
Accuracy after mini-batch    10: 0.319
Accuracy for class 0 after mini-batch    10: 0.137
Accuracy for class 1 after mini-batch    10: 0.206
Accuracy for class 2 after mini-batch    10: 0.517
Accuracy for class 3 after mini-batch    10: 0.435
Loss after mini-batch    15: 1.324
Accuracy after mini-batch    15: 0.377
Accuracy for class 0 after mini-batch    15: 0.098
Accuracy for class 1 after mini-batch    15: 0.492
Accuracy for class 2 after mini-batch    15: 0.626
Accuracy for class 3 after mini-batch    15: 0.302
Loss after mini-batch    20: 1.242
Accuracy after mini-batch    20: 0.464
Accuracy for class 0 after mini-batch    20: 0.528
Accuracy for class 1

NameError: ignored

In [15]:
torch.save(mobilenet_v2.state_dict(), save_path)